In [42]:
import os
# Find the latest version of spark 3.0  from http://www.apache.org/dist/spark/ and enter as the spark version
# For example:
# spark_version = 'spark-3.0.3'
spark_version = 'spark-3.3.1'
os.environ['SPARK_VERSION']=spark_version

# Install Spark and Java
!apt-get update
!apt-get install openjdk-8-jdk-headless -qq > /dev/null
!wget -q https://downloads.apache.org/spark/spark-3.3.1/spark-3.3.1-bin-hadoop3.tgz
!tar xf $SPARK_VERSION-bin-hadoop3.tgz
!pip install -q findspark

# Set Environment Variables
os.environ["JAVA_HOME"] = "/usr/lib/jvm/java-8-openjdk-amd64"
os.environ["SPARK_HOME"] = f"/content/{spark_version}-bin-hadoop3"

# Start a SparkSession
import findspark
findspark.init()

Hit:1 https://cloud.r-project.org/bin/linux/ubuntu bionic-cran40/ InRelease
Ign:2 https://developer.download.nvidia.com/compute/machine-learning/repos/ubuntu1804/x86_64  InRelease
Hit:3 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu1804/x86_64  InRelease
Hit:4 https://developer.download.nvidia.com/compute/machine-learning/repos/ubuntu1804/x86_64  Release
Hit:5 http://security.ubuntu.com/ubuntu bionic-security InRelease
Hit:6 http://ppa.launchpad.net/c2d4u.team/c2d4u4.0+/ubuntu bionic InRelease
Hit:8 http://ppa.launchpad.net/cran/libgit2/ubuntu bionic InRelease
Hit:9 http://archive.ubuntu.com/ubuntu bionic InRelease
Hit:10 http://archive.ubuntu.com/ubuntu bionic-updates InRelease
Hit:11 http://ppa.launchpad.net/deadsnakes/ppa/ubuntu bionic InRelease
Hit:12 http://archive.ubuntu.com/ubuntu bionic-backports InRelease
Hit:13 http://ppa.launchpad.net/graphics-drivers/ppa/ubuntu bionic InRelease
Reading package lists... Done


In [43]:
!wget https://jdbc.postgresql.org/download/postgresql-42.2.9.jar

--2022-11-15 01:13:27--  https://jdbc.postgresql.org/download/postgresql-42.2.9.jar
Resolving jdbc.postgresql.org (jdbc.postgresql.org)... 72.32.157.228, 2001:4800:3e1:1::228
Connecting to jdbc.postgresql.org (jdbc.postgresql.org)|72.32.157.228|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 914037 (893K) [application/java-archive]
Saving to: ‘postgresql-42.2.9.jar.2’

postgresql-42.2.9.j 100%[===================>] 892.61K  5.77MB/s    in 0.2s    

2022-11-15 01:13:28 (5.77 MB/s) - ‘postgresql-42.2.9.jar.2’ saved [914037/914037]



In [44]:
from pyspark.sql import SparkSession
spark = SparkSession.builder.appName("CloudETL").config("spark.driver.extraClassPath","/content/postgresql-42.2.9.jar").getOrCreate()

In [45]:
from pyspark import SparkFiles
# Load in employee.csv from S3 into a DataFrame
url = "https://s3.amazonaws.com/amazon-reviews-pds/tsv/amazon_reviews_us_Video_Games_v1_00.tsv.gz"
spark.sparkContext.addFile(url)

df = spark.read.csv(SparkFiles.get("amazon_reviews_us_Video_Games_v1_00.tsv.gz"), sep="\t", header=True, inferSchema=True)
df.show(10)

+-----------+-----------+--------------+----------+--------------+--------------------+----------------+-----------+-------------+-----------+----+-----------------+--------------------+--------------------+-------------------+
|marketplace|customer_id|     review_id|product_id|product_parent|       product_title|product_category|star_rating|helpful_votes|total_votes|vine|verified_purchase|     review_headline|         review_body|        review_date|
+-----------+-----------+--------------+----------+--------------+--------------------+----------------+-----------+-------------+-----------+----+-----------------+--------------------+--------------------+-------------------+
|         US|   12039526| RTIS3L2M1F5SM|B001CXYMFS|     737716809|Thrustmaster T-Fl...|     Video Games|          5|            0|          0|   N|                Y|an amazing joysti...|Used this for Eli...|2015-08-31 00:00:00|
|         US|    9636577| R1ZV7R40OLHKD|B00M920ND6|     569686175|Tonsee 6 buttons ...| 

In [46]:
# Converting timestamp type to date type for required column
from pyspark.sql.functions import *
df = df.withColumn("review_date", to_date(col("review_date"),"yyyy-MM-dd"))
df.show(10)
df.printSchema()

+-----------+-----------+--------------+----------+--------------+--------------------+----------------+-----------+-------------+-----------+----+-----------------+--------------------+--------------------+-----------+
|marketplace|customer_id|     review_id|product_id|product_parent|       product_title|product_category|star_rating|helpful_votes|total_votes|vine|verified_purchase|     review_headline|         review_body|review_date|
+-----------+-----------+--------------+----------+--------------+--------------------+----------------+-----------+-------------+-----------+----+-----------------+--------------------+--------------------+-----------+
|         US|   12039526| RTIS3L2M1F5SM|B001CXYMFS|     737716809|Thrustmaster T-Fl...|     Video Games|          5|            0|          0|   N|                Y|an amazing joysti...|Used this for Eli...| 2015-08-31|
|         US|    9636577| R1ZV7R40OLHKD|B00M920ND6|     569686175|Tonsee 6 buttons ...|     Video Games|          5|    

In [47]:
# Printing number of records in dataset
df = df.dropna()
df = df.dropDuplicates()
print(df.count())

1785886


In [48]:
# Creating dataframe for review_id_table
review_id_table = df.select(["review_id", "customer_id", "product_id", "product_parent", "review_date"])
review_id_table.printSchema()

review_id_table.show(10)

root
 |-- review_id: string (nullable = true)
 |-- customer_id: integer (nullable = true)
 |-- product_id: string (nullable = true)
 |-- product_parent: integer (nullable = true)
 |-- review_date: date (nullable = true)

+--------------+-----------+----------+--------------+-----------+
|     review_id|customer_id|product_id|product_parent|review_date|
+--------------+-----------+----------+--------------+-----------+
|R1005PB41SDWGY|    1246208|B00HN9LS48|     385183836| 2015-02-08|
| R100AUCA9G6HH|   40693899|B002KSPKRS|      44443896| 2009-10-07|
|R10182UTLL75O0|   37307165|B00130387I|     954736702| 2008-04-14|
|R102L60DYWC7BT|   11494390|B004MKN3XU|     171729072| 2011-08-23|
|R102LT5EV9KD0T|    2391530|B00CMQTUSS|     636573779| 2014-06-26|
|R102OX47XASUO7|   44907426|B00004SVOG|     228547293| 2014-01-06|
|R103F2KVCA5L0I|   23173956|B0016BRKJG|     215559881| 2008-09-08|
|R103TA6UYAME81|   13972517|B00005B8G1|     111876355| 2011-12-17|
|R1045Q4149T2GG|   12803471|B00069ZINY|   

In [49]:
# Creating products table
product_gb = df.groupby("product_id", "product_title").count()

In [50]:
products = product_gb.select("product_id", "product_title")

In [51]:
# Creating customers table
customer_gb = df.groupby("customer_id").count()

In [52]:
customers = customer_gb.withColumnRenamed("count", "customer_count")
customers = customers.withColumn("customer_count", customers.customer_count.cast('int'))
customers.show()

+-----------+--------------+
|customer_id|customer_count|
+-----------+--------------+
|   35350046|             2|
|   44433869|             1|
|      51123|             1|
|    9663853|             2|
|   48707250|             1|
|   44445916|             1|
|    1689312|             1|
|   49576952|             3|
|   48515527|             8|
|   15372644|             9|
|   51115911|             2|
|   33598770|             3|
|   50553272|             1|
|    7455053|            50|
|   41457095|             9|
|    9964795|             1|
|   12286053|             1|
|   33217146|             1|
|    6367756|            65|
|    2613933|             5|
+-----------+--------------+
only showing top 20 rows



In [53]:
# Creating vine table
vine_table = df.select(["review_id", "star_rating", "helpful_votes", "total_votes", "vine"])

In [54]:
# Configure settings for RDS
mode = "append"
jdbc_url=""
config = {"user":"", 
          "password": "", 
          "driver":"org.postgresql.Driver"}

In [56]:
review_id_table.write.jdbc(url=jdbc_url, table='review_id_table1', mode=mode, properties=config)
products.write.jdbc(url=jdbc_url, table='products1', mode=mode, properties=config)
customers.write.jdbc(url=jdbc_url, table='customers1', mode=mode, properties=config)
vine_table.write.jdbc(url=jdbc_url, table='vine_table1', mode=mode, properties=config)